# Analiza prometnih nesreč v Sloveniji

In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
from mapboxgl.viz import *
from mapboxgl.utils import *

In [4]:
# Import data
dnevi=pd.read_csv('data/po_dnevih.csv', sep=";")
udelezenci=pd.read_csv('data/po_vrstah_udelezencev.csv', sep=";")
ceste=pd.read_csv('data/po_vrsti_ceste.csv', sep=";")
meseci=pd.read_csv('data/po_mesecih.csv', sep=";")
regije=pd.read_csv('data/po_regijah.csv', sep=";")
lokacije=pd.read_csv('data/po_lokaciji.csv', sep=";")
ura=pd.read_csv('data/po_urah.csv', sep=";")
alkohol=pd.read_csv('data/alkohol.csv', sep=";")

#### Zamenjava manjkajočih podatkov s povprečjem:

In [5]:
alkohol["Stopnje alkoholiziranosti 0,80 in več"].fillna(alkohol.groupby("Kategorija")["Stopnje alkoholiziranosti 0,80 in več"].transform("mean"), inplace=True)
alkohol["Stopnje alkoholiziranosti 0,80 in več"] = alkohol["Stopnje alkoholiziranosti 0,80 in več"].astype(int)

#### Izračun števila prometnih nesreč po dnevih v tednu, razdeljenih po vrsti nesreče in izris grafa:

In [6]:
# 
dnevi_sum = dnevi.groupby(['Dan v tednu'], as_index = False).agg({'Prometne nesreče s smrtnim izidom': [np.sum],
                                                                  'Prometne nesreče s telesno poškodbo': [np.sum],
                                                                  'Prometne nesreče z materialno škodo': [np.sum],
                                                                  'Prometne nesreče - SKUPAJ': [np.sum]
                                                                 }) 

dnevi_sum.columns = ["dan_v_tednu", "st_smrt", "st_telesna_poskodba", "st_materialna_skoda", "skupaj"] 
dnevi_max=dnevi_sum.iloc[dnevi_sum["skupaj"].idxmax()]["dan_v_tednu"]
dnevi_sum = dnevi_sum.sort_values(by="skupaj", ascending=[False])
dnevi_sum;

In [7]:
dnevi_max

'Petek'

In [8]:
# draws plot
trace1 = go.Bar(
    y=dnevi_sum["dan_v_tednu"],
    x=dnevi_sum["st_smrt"],
    name='Število nesreč s smrtnim izidom',
    orientation = "h",
    marker=dict(
        color='#000000',
    )
    
)
trace2 = go.Bar(
    y=dnevi_sum["dan_v_tednu"],
    x=dnevi_sum["st_telesna_poskodba"],
    name='Število nesreč telesnih poškodb',
    orientation = "h",
    marker=dict(
        color='#FF2222',
    )
)
trace3 = go.Bar(
    y=dnevi_sum["dan_v_tednu"],
    x=dnevi_sum["st_materialna_skoda"],
    name='Število nesreč z materialno škodo',
    orientation = "h",
    marker=dict(
        color='#AAAAAA',
    )
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

#### Izračun števila prometnih nesreč skozi leta in risanje grafa:

In [9]:
# 
smrti_sum = dnevi.groupby(['leto'], as_index = False).agg({'Prometne nesreče s smrtnim izidom': [np.sum],
                                                                  'Prometne nesreče s telesno poškodbo': [np.sum],
                                                                  'Prometne nesreče z materialno škodo': [np.sum],
                                                                  'Prometne nesreče - SKUPAJ': [np.sum]
                                                                 }) 

smrti_sum.columns = ["leto", "st_smrt", "st_telesna_poskodba", "st_materialna_skoda", "skupaj"]    
smrti_sum = smrti_sum.sort_values(by="leto", ascending=[False])
smrti_sum;

In [10]:
# Create a trace
trace = go.Scatter(
    x = smrti_sum["leto"],
    y = smrti_sum["st_smrt"],
    marker=dict(
        color='#000000',
    )
)

data = [trace]
iplot(data, filename='basic-line')

#### Osnutek po regijah razdeljenega zemljevida, ki ga bomo uporabili v kasnejši fazi:

In [11]:
token = "pk.eyJ1IjoiaW1pbG9zayIsImEiOiJjamZ2b2toNTExbjUwMnFwZGs2bjN2c2R4In0.gd6IuS0fih6OpePAt3HmPw"

# create choropleth from polygon features stored as GeoJSON
viz = ChoroplethViz('https://gist.githubusercontent.com/imilosk/47013ab15f8a329b0497853f95d081d5/raw/48d364de4c88b0fc6fbd9e42012b47f87def285d/slovenia.json', 
                     color_property='density',
                     access_token=token,
                     color_stops=create_color_stops([0, 50, 100, 500, 1500], colors='YlOrRd'),
                     color_function_type='interpolate',
                     line_stroke='--',
                     line_color='#ff3300',
                     line_width=1,
                     opacity=0.5,
                     center=(14.9955, 46.1512),
                     zoom=7,
                     below_layer='waterway-label'
                    )
viz.show()

#### Izračun in izris podatkov o alkoholiziranosti različnih udeležencev prometa:

In [12]:
# 
alko_sum = alkohol.groupby(['Kategorija'], as_index = False).agg({'Stopnje alkoholiziranosti 0,50 in več': [np.sum],
                                                                  'Stopnje alkoholiziranosti 0,80 in več': [np.sum],
                                                                  'Stopnje alkoholiziranosti 1,50 in več': [np.sum],
                                                                  'Število alkoholiziranih udeležencev - SKUPAJ': [np.sum]
                                                                 }) 

alko_sum.columns = ["kategorija","alko_05", "alko_08", "alko_15", "skupaj"]

alko_sum = alko_sum.sort_values(by="skupaj", ascending=[False])
alko_sum = alko_sum.drop([0, 3, 9])
#alko_max = alko_sum["kategorija"][alko_sum["skupaj"].idxmax()]["kategorija"]
alko_max=alko_sum["kategorija"][8]

In [13]:
trace1 = go.Bar(
    x=alko_sum["kategorija"],
    y=alko_sum["alko_05"],
    name='Stopnje alkoholiziranosti 0,50 in več'
)
trace2 = go.Bar(
    x=alko_sum["kategorija"],
    y=alko_sum["alko_08"],
    name='Stopnje alkoholiziranosti 0,80 in več'
)
trace3 = go.Bar(
    x=alko_sum["kategorija"],
    y=alko_sum["alko_15"],
    name='Stopnje alkoholiziranosti 1,50 in več'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')

#### Izračun porazdelitve prometnih nesreč skozi dan:

In [14]:
ura_avg = ura.groupby(['Ura v dnevu'], as_index = False).agg({'Prometne nesreče - SKUPAJ': [np.sum]}) 

ura_avg.columns = ["Ura v dnevu","sum"]
ura_max=ura_avg.iloc[ura_avg["sum"].idxmax()]["Ura v dnevu"]

ura_avg["sum"]=ura_avg["sum"]/(365*11+3)

In [15]:
# Create a trace
trace = go.Bar(
    x = ura_avg["Ura v dnevu"],
    y = ura_avg["sum"],
    marker=dict(
        color='#2b8cbe',
    )
)

data = [trace]
iplot(data, filename='basic-bar')

In [16]:
# 
lok_sum = lokacije.groupby(["Lokacija"], as_index = False).agg({'Prometne nesreče - SKUPAJ': [np.sum]}) 

lok_sum.columns = ["kje","skupaj"]

lok_sum
lok_max = lok_sum.iloc[lok_sum["skupaj"].idxmax()+1]["kje"]
lok_max

'V naselju'

In [17]:
mes_sum = meseci.groupby(["Mesec v letu"], as_index = False).agg({'Prometne nesreče - SKUPAJ': [np.sum]}) 

mes_sum.columns = ["mesec","sum"]

mes_sum
mes_max = mes_sum.iloc[mes_sum["sum"].idxmax()]["mesec"]
mes_max

'Oktober'

In [18]:
reg_sum = regije.groupby(["Regija"], as_index = False).agg({'Prometne nesreče - SKUPAJ': [np.sum]}) 

reg_sum.columns = ["regija","sum"]

reg_sum
reg_max = reg_sum.iloc[reg_sum["sum"].idxmax()]["regija"]
reg_max

'Osrednjeslovenska'

In [19]:

udl_sum = udelezenci.groupby(["Udeleženci"], as_index = False).agg({'Število udeležencev - SKUPAJ': [np.sum]}) 

udl_sum.columns = ["udel","sum"]

udl_sum
udl_max = udl_sum.iloc[udl_sum["sum"].idxmax()]["udel"]
udl_max

'Vrste udeležencev - SKUPAJ'

In [20]:
#alkohol, dan, kje, mesec, regija, ura, kdo, cesta
print(alko_max+" , "+dnevi_max+" , "+lok_max+" , "+ mes_max+ " , "+reg_max+" , ",ura_max," , "+udl_max)

Vozniki osebnih avtomobilov , Petek , V naselju , Oktober , Osrednjeslovenska ,  15  , Vrste udeležencev - SKUPAJ
